In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd


### 더본코리아 서울특별시 음식점 정보 수집 크롤러 제작
#### 수집 데이터 목록
- 브랜드명
- 도시명
- 지점명
- 도로명주소
- 전화번호
- 영업시간
- 매장입구이미지

In [12]:
def theborn_store_info_crawler():
    driver = webdriver.Chrome("C:/chromedriver_win32/chromedriver.exe")
    time.sleep(1)

    # 더본 사이트 들어가기
    driver.get("http://www.theborn.co.kr/store/domestic-store/")
    time.sleep(1)

    # 서울특별시 매장만 보기
    driver.find_element(By.CSS_SELECTOR,"#depth1 [value = '18']").click()
    time.sleep(1)

    # 1페이지 html 소스 가져와서 soup에 저장
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')

    # 총 페이지 수 total_page에 저장
    total_page = soup.select("#pagination > li")[-1].get("data-page")


    # 1페이지부터 크롤링 시작
    cur_page = 1

    # 페이지 html 소스 가져와서 soup에 저장
    store_list = []

    brand = []
    city = []
    store = []
    address = []
    telephone = []
    openhour = []
    storeimg = []

    while cur_page < int(total_page)+1:
        req = driver.page_source
        soup=BeautifulSoup(req, 'html.parser')

        brand_html = soup.select("#store_list > div > ul > li:nth-child(1) > span")
        city_html = soup.select("#store_list > div > ul > li:nth-child(2)")
        store_html = soup.select("#store_list > div > ul > li:nth-child(3)")
        address_html = soup.select("#store_list > div > ul > li:nth-child(4)")
        telephone_html = soup.select("#store_list > div > ul > li:nth-child(5)")
        openhour_html = soup.select("#store_list > div > div > div > table > tbody > tr:nth-child(2) > td > span")
        storeimg_html = soup.select("#store_list > div > div > div > div > p > img")

        for i in range(len(brand_html)):
            store = {}
            store["brand"] = 0
            store["city"] = 0
            store["store"] = 0
            store["address"] = 0
            store["telephone"] = 0
            store["openhour"] = 0
            store["img"] = 0
            
            
            store["brand"] = soup.select("#store_list > div:nth-child({}) > ul > li:nth-child(1) > span".format(i+1))[0].get_text()
            store["city"] = soup.select("#store_list > div:nth-child({}) > ul > li:nth-child(2)".format(i+1))[0].get_text()
            store["store"] = soup.select("#store_list > div:nth-child({}) > ul > li:nth-child(3)".format(i+1))[0].get_text()
            store["address"] = soup.select("#store_list > div:nth-child({}) > ul > li:nth-child(4)".format(i+1))[0].get_text()
            store["telephone"] = soup.select("#store_list > div:nth-child({}) > ul > li:nth-child(5)".format(i+1))[0].get_text()
            store["openhour"] = soup.select("#store_list > div:nth-child({}) > div > div > table > tbody > tr:nth-child(2) > td > span".format(i+1))[0].get_text()

            if len(soup.select("#store_list > div:nth-child({}) > div > div > div > p > img".format(i+1))) > 0 :
                store["img"] = soup.select("#store_list > div:nth-child({}) > div > div > div > p > img".format(i+1))[0].get("src")
                store_list.append(store)
            else : 
                store_list.append(store)


        if cur_page == 1:
            driver.find_element(By.CSS_SELECTOR,"#pagination > li:nth-child({})".format(2)).click()
            cur_page +=1
            time.sleep(1)

        elif cur_page == 2:
            driver.find_element(By.CSS_SELECTOR,"#pagination > li:nth-child({})".format(4)).click()
            cur_page +=1
            time.sleep(1)

        elif cur_page == int(total_page):
            cur_page +=1

        else :
            driver.find_element(By.CSS_SELECTOR,"#pagination > li:nth-child({})".format(6)).click()
            cur_page +=1
            time.sleep(1)


    location_df = pd.DataFrame(store_list)

    #데이터프레임 저장하기
    #location_df.to_csv("C:/dataset/theborn_restaurant_information.csv",index = False, encoding="utf-8-sig")
    location_df
